In [1]:
import os
import sys

sys.path.append('/home/robinm/phd/problog/')

In [2]:
from problog.program import PrologString
from problog.prolog_engine.translate import TranslatedProgram, translate_clausedb
from problog.engine import DefaultEngine
from problog.logic import Term, Var
from problog.prolog_engine.engine_prolog import EngineProlog
from problog.formula import LogicFormula, LogicDAG
from graphviz import Source
from problog.cycles import break_cycles
from problog.sdd_formula import SDD

In [7]:
file='''
0.5::edge(1,2).
0.6::edge(1,3).
0.9::edge(2,4).
0.9::edge(2,5).
0.6::edge(3,6).
0.9::edge(4,6).
0.9::edge(5,6).
path(X,Y) :- edge(X,Y).
path(X,Y) :- edge(Z,Y), path(X,Z).
query(path(1,6)).
'''

In [8]:
program = PrologString(file)
engine = EngineProlog()
db = engine.prepare(program)
transformed = translate_clausedb(db)
for l in transformed.get_lines():
    print(l+'.')

ad([p(0.5,edge(1,2),1)],[]).
ad([p(0.6,edge(1,3),2)],[]).
ad([p(0.9,edge(2,4),3)],[]).
ad([p(0.9,edge(2,5),4)],[]).
ad([p(0.6,edge(3,6),5)],[]).
ad([p(0.9,edge(4,6),6)],[]).
ad([p(0.9,edge(5,6),7)],[]).
ad([p(1.0,path(A1,A2),8)],[edge(A1,A2)]).
ad([p(1.0,path(A1,A2),9)],[edge(A3,A2),path(A1,A3)]).
ad([p(1.0,query(path(1,6)),10)],[]).


# Standard

In [9]:
ground = engine.ground_all(db, target=LogicFormula())

In [10]:
ac = SDD.create_from(ground)

In [11]:
ac.evaluate()

{path(1,6): 0.668448}

# K-best